In [98]:
from CC.trainer import NERTrainer

args = {
    'num_epochs': 30,
    'num_gpus': [0, 1, 2, 3],
    'bert_config_file_name': './model/chinese_wwm_ext/config.json',
    'pretrained_file_name': './model/chinese_wwm_ext/pytorch_model.bin',
    'hidden_dim': 300,
    'max_seq_length': 150,
    'max_scan_num': 1000000,
    'train_file': 'data/ontonotes5/train.json',
    'eval_file': 'data/ontonotes5/dev.json',
    'test_file': 'data/ontonotes5/test.json',
    'bert_vocab_file': './model/chinese_wwm_ext/vocab.txt',
    'tag_file': 'data/ontonotes5/ontonotes5_labels.txt',
    'output_eval': True,
    'loader_name': 'le_loader',
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "default_tag": "O",
    'batch_size': 64,
    'eval_batch_size': 512,
    'do_shuffle': True,
    'model_name': 'LEBert',
    'task_name': 'super_predict_model'
}

from CC.predicter import NERPredict
import json

# 使用了预训练模型
args["lstm_crf_model_file"] = "/home/lpc/repos/ccNERx/save_model/ontonotes5/lstm_crf/lstm_crf_13823.pth"
args["bert_model_file"] = "/home/lpc/repos/ccNERx/save_model/ontonotes5/LEBert/LEBert_13823.pth"
predict = NERPredict(**args)


calculate data/ontonotes5/train.json etag: 100%|██████████| 16.3M/16.3M [00:00<00:00, 349MB/s]
calculate data/ontonotes5/dev.json etag: 100%|██████████| 2.01M/2.01M [00:00<00:00, 354MB/s]
calculate data/ontonotes5/test.json etag: 100%|██████████| 2.00M/2.00M [00:00<00:00, 344MB/s]
calculate data/ontonotes5/ontonotes5_labels.txt etag: 100%|██████████| 614/614 [00:00<00:00, 1.21MB/s]

kwargs parser: {
    "use_gpu": false,
    "loader_name": "le_loader",
    "model_name": "LEBert",
    "lstm_crf_model_file": "/home/lpc/repos/ccNERx/save_model/ontonotes5/lstm_crf/lstm_crf_13823.pth",
    "bert_model_file": "/home/lpc/repos/ccNERx/save_model/ontonotes5/LEBert/LEBert_13823.pth",
    "hidden_dim": 300,
    "bert_vocab_file": "./model/chinese_wwm_ext/vocab.txt",
    "bert_config_file_name": "./model/chinese_wwm_ext/config.json",
    "tag_file": "data/ontonotes5/ontonotes5_labels.txt",
    "padding_length": 512,
    "num_gpus": [
        0,
        1,
        2,
        3
    ]
}
kwargs parser: {
    "batch_size": 64,
    "eval_batch_size": 512,
    "test_batch_size": 16,
    "word_embedding_file": "./data/tencent/word_embedding.txt",
    "word_vocab_file": "./data/tencent/tencent_vocab.txt",
    "train_file": "data/ontonotes5/train.json",
    "eval_file": "data/ontonotes5/dev.json",
    "test_file": "data/ontonotes5/test.json",
    "tag_file": "data/ontonotes5/ontonotes5


count line size data/ontonotes5/ontonotes5_labels.txt: 74L [00:00, 485725.35L/s]
build line mapper: 74L [00:00, 520769.29L/s] [00:00<?, ?it/s]
load vocab from list: 100%|██████████| 73/73 [00:00<00:00, 440552.79it/s]

load cached ./temp/476f67c99ca56e0b89056f7ef462d2ac-4_ce61c1b52833ada8b5f3ec3bfaf00bb7_a837adf46522abcbc2acc8c9f9ea7d60_6aa179f1cd1584c53fef6462f9143bca/1000000/matched_words
load cached ./temp/476f67c99ca56e0b89056f7ef462d2ac-4_ce61c1b52833ada8b5f3ec3bfaf00bb7_a837adf46522abcbc2acc8c9f9ea7d60_6aa179f1cd1584c53fef6462f9143bca/1000000/word_vocab
load cached ./temp/476f67c99ca56e0b89056f7ef462d2ac-4_ce61c1b52833ada8b5f3ec3bfaf00bb7_a837adf46522abcbc2acc8c9f9ea7d60_6aa179f1cd1584c53fef6462f9143bca/1000000/vocab_embedding



/home/yfy/miniconda3/envs/ccner/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1642: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


Load pretrained embedding from file.........


In [101]:
import torch
text = "九十年代"
print(predict([text]))
hidden_state,masks = predict.p([text])

crf = predict.birnncrf.crf
features,masks = predict.birnncrf._BiRnnCrf__build_features(hidden_state,masks)

features = crf.fc(features)
L = features.size(1)
masks_ = masks[:, :L].float()

tag_vocab = predict.dataloader()["tag_vocab"]
labels = tag_vocab.item2idx.keys()
labels_set = set()
for label in labels:
    labels_set.add(label.split("-")[-1])

from CC.loaders.utils.label import get_labels
# 构造标签集合

ans = []
for label in labels_set:
    paths = []
    if label != "O":
        paths = ["O"] + get_labels(label,len(text),has_end=False,middle_symbol="I")
    paths += ["O"] * (150-len(paths))
    # print(paths)
    paths = torch.tensor(tag_vocab.token2id(paths),dtype=torch.long).unsqueeze(0)
    ans.append(crf._CRF__score_sentence(features,paths[:, :L].long(),masks_).item())
    # print(crf.loss(features,paths,masks))
soft = torch.nn.Softmax(-1)
ans = soft(torch.tensor(ans))
for label,ans in zip(labels_set,ans):
    print(label,round(ans.item(),6))

[(['九', '十', '年', '代'], ['B-DATE', 'I-DATE', 'I-DATE', 'E-DATE'])]
QUANTITY 0.0
LOC 0.0
LAW 0.0
CARDINAL 0.0
PRODUCT 0.0
GPE 0.0
DATE 0.999959
ORDINAL 0.0
MONEY 0.0
ORG 0.0
NORP 0.0
EVENT 0.0
PERSON 0.0
PERCENT 0.0
FAC 0.0
WORK_OF_ART 0.0
TIME 6e-06
LANGUAGE 0.0
O 3.5e-05
